In [ ]:
import os
import pandas as pd
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
# 0. 파일 경로 설정
input_data_file = "df_master_TM_proj_lemma.csv"
market_json_file = "market_lexicon.json"
dictionary_json_file = "sentiment_lexicon.json"
output_file = "final_tone_analysis_result.csv"

In [27]:
# 1. 데이터 로드
print(">>> 1. 데이터 로드 중...")
df = pd.read_csv(
    input_data_file,
    engine='python',
    on_bad_lines='skip'
)
df = df.dropna(subset=['cleaned_text_lemma', 'doc_id'])
print(f"   - 분석 대상 문장(행) 개수: {len(df)}개")


>>> 1. 데이터 로드 중...
   - 분석 대상 문장(행) 개수: 171788개


In [28]:
# 2. 문장 톤 계산 함수 (시장/사전 공용)

def calculate_sentence_tone(df_input, vocab, hawkish_w, dovish_w):
    # 1. 벡터화 (문장별 단어 빈도 계산)
    vectorizer = CountVectorizer(vocabulary=vocab, ngram_range=(1, 5))
    dtm = vectorizer.transform(df_input['cleaned_text_lemma'])

    # 2. 문장별 점수 합산 (빈도 * 가중치)
    sent_h_score = dtm.dot(hawkish_w)
    sent_d_score = dtm.dot(dovish_w)
    total_score = sent_h_score + sent_d_score

    # 3. 문장 톤 계산
    with np.errstate(divide='ignore', invalid='ignore'):
        sent_tones = (sent_h_score - sent_d_score) / total_score
    return np.nan_to_num(sent_tones, 0) # NaN -> 0

In [29]:
# 3. [시장 접근법] 문장 -> 문서 어조 계산

print("\n>>> 3. 시장 접근법(Market Approach) 계산 시작...")

if os.path.exists(market_json_file):
    # (1) JSON 로드
    with open(market_json_file, "r", encoding="utf-8") as f:
        market_lexicon = json.load(f)

    # (2) 단어장 및 가중치 정렬 (Vectorizer 순서에 맞추기 위해 필수)
    m_vocab_raw = list(market_lexicon.keys())

    # 순서 보장을 위해 임시 벡터라이저 생성
    temp_vec = CountVectorizer(vocabulary=m_vocab_raw, ngram_range=(1, 5))
    feature_names = temp_vec.get_feature_names_out()

    m_h_weights = []
    m_d_weights = []

    for word in feature_names:
        info = market_lexicon.get(word, {})
        pol = info.get('polarity')
        score = info.get('score', 0)

        if pol == 'hawkish':
            m_h_weights.append(score)
            m_d_weights.append(0)
        elif pol == 'dovish':
            m_h_weights.append(0)
            m_d_weights.append(score)
        else:
            m_h_weights.append(0)
            m_d_weights.append(0)

    # (3) 문장 톤 계산 함수 호출
    # (feature_names가 정렬된 단어장이므로 vocab 자리에 넣습니다)
    df['Sent_Tone_Market'] = calculate_sentence_tone(
        df, feature_names, np.array(m_h_weights), np.array(m_d_weights)
    )

    # (4) 문장 라벨링 (1: 매파, -1: 비둘기, 0: 중립)
    df['Sent_Label_Market'] = np.sign(df['Sent_Tone_Market'])

    print("   ✅ 시장 접근법 문장 분석 완료")

else:
    print("   ⚠️ 시장 접근법 사전 파일이 없어 0으로 처리합니다.")
    df['Sent_Tone_Market'] = 0
    df['Sent_Label_Market'] = 0


>>> 3. 시장 접근법(Market Approach) 계산 시작...
   ✅ 시장 접근법 문장 분석 완료


In [32]:
print("\n>>> 4. 사전 접근법(Dictionary Approach) 계산 시작...")

if os.path.exists(dictionary_json_file):
    # (1) JSON 로드 (시장 접근법과 동일하게 변경)
    with open(dictionary_json_file, "r", encoding="utf-8") as f:
        dict_map = json.load(f)

    print(f"   - 사전 로드 완료! 포함된 단어 수: {len(dict_map)}개")

    # (2) 가중치 배열 생성
    # JSON 구조가 {"word": {"polarity": "hawkish", "score": 1.5}, ...} 라고 가정
    d_vocab_raw = list(dict_map.keys())

    # 순서 보장을 위해 임시 벡터라이저 생성
    temp_vec_d = CountVectorizer(vocabulary=d_vocab_raw, ngram_range=(1, 5))
    feat_d = temp_vec_d.get_feature_names_out()

    d_h_weights = []
    d_d_weights = []

    for word in feat_d:
        info = dict_map.get(word, {})

        # 키 이름이 시장 접근법과 같다면 'polarity', 다르다면 'label' 등으로 수정 필요
        p = info.get('polarity', '') # 혹은 info.get('label', '')
        s = info.get('score', 1.0)   # 점수가 없으면 기본값 1.0

        if 'hawkish' in p.lower():
            d_h_weights.append(s)
            d_d_weights.append(0)
        elif 'dovish' in p.lower():
            d_h_weights.append(0)
            d_d_weights.append(s)
        else:
            d_h_weights.append(0)
            d_d_weights.append(0)

    # (3) 문장 톤 계산 (함수는 위에서 정의한 것 재사용)
    df['Sent_Tone_Dict'] = calculate_sentence_tone(
        df, feat_d, np.array(d_h_weights), np.array(d_d_weights)
    )
    df['Sent_Label_Dict'] = np.sign(df['Sent_Tone_Dict'])
    print("   ✅ 사전 접근법 문장 분석 완료")

else:
    print(f"   ⚠️ '{dictionary_json_file}' 파일이 없어 사전 접근법은 0으로 처리합니다.")
    df['Sent_Tone_Dict'] = 0
    df['Sent_Label_Dict'] = 0


>>> 4. 사전 접근법(Dictionary Approach) 계산 시작...
   - 사전 로드 완료! 포함된 단어 수: 15000개
   ✅ 사전 접근법 문장 분석 완료


In [34]:
# 5. 문서 단위 집계 및 저장

print("\n>>> 5. 문서별 최종 톤 집계 중...")

# 집계 함수: (매파 문장 수 - 비둘기 문장 수) / (매파 문장 수 + 비둘기 문장 수)
def aggregate_tone(label_series):
    h_count = (label_series > 0).sum()
    d_count = (label_series < 0).sum()
    total = h_count + d_count

    if total == 0: return 0
    return (h_count - d_count) / total

# doc_id 기준 그룹화
final_df = df.groupby('doc_id').agg({
    'Sent_Label_Market': aggregate_tone,
    'Sent_Label_Dict': aggregate_tone
}).rename(columns={
    'Sent_Label_Market': 'Market_Tone',
    'Sent_Label_Dict': 'Dictionary_Tone'
}).reset_index()

# 저장
final_df.to_csv(output_file, index=False)

print(f"\n=======================================================")
print(f"🎉 분석 완료! 결과 파일: {output_file}")
print(f"=======================================================")
print(final_df.head())


>>> 5. 문서별 최종 톤 집계 중...

🎉 분석 완료! 결과 파일: final_tone_analysis_result.csv
          doc_id  Market_Tone  Dictionary_Tone
0  FOMC_20130918     0.392405        -0.975904
1  FOMC_20131030     0.466667        -0.914062
2  FOMC_20131218     0.328358        -0.842294
3  FOMC_20140129     0.596730        -0.907455
4  FOMC_20140319     0.455224        -0.839416
